In [65]:
# !pip install geocoder
# !pip3 install nltk
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import category_encoders as ce # transformers for encoding categorical variables into numeric with different techniques
import ast # abstract syntax tree

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import geocoder
import geopy.distance

# анализ текста
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [66]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [117]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [118]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [119]:
# Просмотрим на наши данные и их состав
data.info()

## Поработаем с географическими данными отелей
#### Видим что в данных долготы и широты есть пропуски, заполним их с помощью библиотеки `geocoder`

In [120]:
fill_lat = data.groupby(['hotel_address'], as_index = False)['lat'].median()
fill_lat.rename(columns={'lat':'fill_lat'}, inplace=True)
null_lat=fill_lat[fill_lat.isna().any(axis=1)] #получаем список отелей с пропуском географической широты
# заполняем пропуски с помощью geocoder
null_lat['fill_lat']=null_lat['hotel_address'].apply(lambda x: geocoder.opencage(x,  key='924f41fc864d4bed8c9b438bff7c67e1',timeout=None ).json['lat'])
data = data.merge(null_lat, on=['hotel_address'], how = 'left') # вовзращаем значения в data
data['lat']=data['lat'].fillna(data['fill_lat'])
#null_lat

# аналогично для долготы
fill_lng = data.groupby(['hotel_address'], as_index = False)['lng'].median()
fill_lng.rename(columns={'lng':'fill_lng'}, inplace=True)
null_lng=fill_lng[fill_lng.isna().any(axis=1)] 
null_lng['fill_lng']=null_lng['hotel_address'].apply(lambda x: geocoder.opencage(x,  key='924f41fc864d4bed8c9b438bff7c67e1', timeout=None).json['lng'])
data = data.merge(null_lng, on=['hotel_address'], how = 'left')
data['lng']=data['lng'].fillna(data['fill_lng'])
#null_lng

In [121]:
# Удалим прмежуточный данные
data = data.drop(['fill_lat', 'fill_lng'], axis=1)

In [122]:
# Видиим что пропусков в данных больше нету
data.info()

### Работаем с адресом отеля

In [123]:
# Получаем название город из адреса
def get_city(address):
    if address.split()[-1] == 'Kingdom':
        return address.split()[-5]
    else:
        return address.split()[-2]
    
data['city'] = data['hotel_address'].apply(get_city)

In [124]:
# Выведем количество отелей в каждом городе
data['city'].value_counts()

In [125]:
# Так как колчиство городов не большое их можно закодировать, кодируем города
encoder = ce.OneHotEncoder(cols=['city'])
type_bin = encoder.fit_transform(data['city'])
data = pd.concat([data, type_bin], axis=1)

### Поработаем со страной рецензента

In [126]:
# Преобразуем страну рецензента. Для этого оставим только страны,которые составляют хотя
# бы 1 процент от всего количества.
reviewer_nationality_percent = data['reviewer_nationality'].value_counts() \
                               / data['reviewer_nationality'].count() * 100
reviewer_nationality_percent = reviewer_nationality_percent[reviewer_nationality_percent > 1]
data['reviewer_nationality_new'] = data['reviewer_nationality'] \
                                   .apply(lambda x: x if x in reviewer_nationality_percent else ' Other ')

In [127]:
data['reviewer_nationality_new'].value_counts()

In [128]:
# кодируем страну с помощью  метода LabelEncode
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
data['reviewer_nationality'] = LabelEncoder().fit_transform(data['reviewer_nationality'])
data.head(3)

### Работа с датами отзывов

In [129]:
# Так загруженность отеля зависит от сезона, то скорее всего этот параметр будет влиять на оценку отеля, порабатаем с этим параметром. Извлечем месяц из даты отзыва
# Переводим нужную нам информацию в формат datatime
data['review_date'] = pd.to_datetime(data['review_date'])
data['month'] = data['review_date'].dt.month

In [130]:
# Удаляем промежуточные данные
data = data.drop(['review_date'], axis=1)

In [131]:
# Преобразуем месяц в сезон
def get_season(m):
    if (1 <= m <= 2) or m == 12:
        return "winter"
    if 3 <= m <= 5:
        return "spring"
    if 6 <= m <= 8:
        return "summer"
    if 9 <= m <= 11:
        return "autumn"

data['season'] = data['month'].apply(get_season)

In [132]:
# кодируем сезон
encoder = ce.OneHotEncoder(cols=['season'])
season_bin = encoder.fit_transform(data['season'])
data = pd.concat([data, season_bin], axis=1)

In [133]:
# Преобразуем строку с количеством дней прошедших с последней оценки в число
data['review_update'] = data['days_since_review'].apply(lambda s: s.split()[0])
data['review_update'] = data['review_update'].astype('int')

# удаляем признак days_since_review
data = data.drop(['days_since_review'], axis=1)

### Работа с отзывами

In [134]:
# Анализ отзывов negative_review и positive_review
nltk.downloader.download('vader_lexicon')
sent_analyzer = SentimentIntensityAnalyzer()

# анализ позитивных отзывов
data['pos_review'] = data['positive_review'].apply(lambda s: sent_analyzer.polarity_scores(s)['pos'])

# анализ негативных отзывов
data['neg_review'] = data['negative_review'].apply(lambda s: sent_analyzer.polarity_scores(s)['neg'])

In [135]:
# Удаляем промежуточные данные
data = data.drop(['negative_review', 'positive_review'], axis=1)

In [136]:
# Подсичтаем количество уникальных значений 
data.nunique(dropna=False)

In [137]:
# Построим матрицу корреляции параметров
plt.rcParams['figure.figsize'] = (20,15)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [138]:
# удаляем признак additional_number_of_scoring из-за высокой корреляции с 
# признаком total_number_of_reviews
data = data.drop(['additional_number_of_scoring'], axis=1)

### Поработаем с тэгами

In [139]:
#представление тегов списком
def tags(col):
    res = []
    tag_split = col.split(',')
    for tag in tag_split:
        reg = re.compile('[^a-zA-Z0-9 ]')
        res.append(reg.sub('', tag).strip())
    return res
data['new_tags'] = data['tags'].apply(tags)

In [140]:
# извлечем продолжительность пребывания в отеле
def night_number(col):
    for tag in col:
        if tag.split(' ')[0] == 'Stayed':
            return int(tag.split(' ')[1])
data['night_number'] = data['new_tags'].apply(night_number)
data['night_number'] = data['night_number'].fillna(data['night_number'].median())
data['night_number'] = data['night_number'].apply(lambda x: x if x <= 7 else 10)

In [141]:
# теги описание цели поездки
conditions = ['Couple','Solo traveler','Business trip','Family with young children','Group','Family with older children','With a pet']

data['Couple']=''
data['Solo traveler']=''
data['Business trip']=''
data['Family with young children'] = ''
data['Group']=''
data['Family with older children']=''
data['With a pet']=''

for col in data[conditions].columns:
    data[col]=data['new_tags'].apply(lambda x: 1 if col in x else 0)

In [142]:
# извлечем тип комнаты
def room_type(col):
    for tag in col:
        if tag.split(' ')[-1] == 'Room':
            return tag[:-4]
data['room_type'] = data['new_tags'].apply(room_type)

room_type =(data['room_type'].value_counts(normalize=True).nlargest(15))
data['room_type'] = data['room_type'].apply(lambda x: x.strip() if x in room_type else 'other')
room_type*100

In [143]:
# количество тегов
data['tags_length'] = data['new_tags'].apply(lambda x: len(x))

In [144]:
# удаляем признаки, созданные при обработке тегов
data = data.drop(['tags', 'new_tags', 'room_type'], axis=1)

In [145]:
# убираем признаки которые не обработали, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

## Нормализация данных

In [146]:
data.info()

In [147]:
# список численных признаков, требующих внимания
cols = ['review_total_negative_word_counts',
        'review_total_positive_word_counts',
        'total_number_of_reviews_reviewer_has_given', 
        'total_number_of_reviews',
        'average_score'
       ]
data[cols].hist(figsize=(20, 8))

In [113]:
### логарифмируем признаки

data['review_total_positive_word_counts'] = np.log(data['review_total_positive_word_counts']+ 1)

data['review_total_negative_word_counts'] = np.log(data['review_total_negative_word_counts']+ 1)

data['total_number_of_reviews_reviewer_has_given'] = np.log(data['total_number_of_reviews_reviewer_has_given'])

data['total_number_of_reviews'] = np.log(data['total_number_of_reviews'])

# скорректируем выброс в average_score
data['average_score']=data['average_score'].apply(lambda x: 6.5 if x == 5.2 else x)

In [153]:
# результат
data[cols].hist(figsize=(20, 8))

Теперь модель можно обучать, т.к. в данных остались только числовые признаки

In [154]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [155]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [156]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [157]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [158]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [159]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [160]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))

In [161]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [162]:
test_data.sample(10)

In [163]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [164]:
sample_submission.head(5)

In [165]:
predict_submission = model.predict(test_data)

In [166]:
predict_submission

In [167]:
list(sample_submission)

In [168]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)

In [169]:
sample_submission.head(10)